<a name="ancora"></a>
# Case IA e Visão Computacional - Treinamento

No presente notebook, encontra-se o script usado para o treinamento do modelo de classificação de imagens de árvores e imagens de solo. Um sumário do notebook pode ser visto a seguir.

## Sumário
1. [Inicializacao](#inicializacao)
2. [Organização e Limpeza dos dados](#organizacao)
3. [*Feature Engineering*](#feat_eng)
4. [Treinamento do modelo (CNN)](#CNN)

## Inicialização <a name="inicializacao"></a>

Importando bibliotecas básicas:

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

Importando os dados (*.zip*):

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Data.zip to Data.zip


In [3]:
import shutil
zipfile = '/content/Data.zip'
unzip_to = '/content/XMB/'
shutil.unpack_archive(zipfile, unzip_to)

In [4]:
CATEGORIES = ["soil","tree"]
DATADIR = "/content/XMB/Data"

# lista com o nome das imagens
imgs_list = os.listdir(DATADIR)

[Voltar ao topo](#ancora)

## Organização e Limpeza dos dados <a name="organizacao"></a>

Permitindo verificar se os dados estão desbalanceados ou não, a função a seguir conta a quantidade de amostras em cada categoria. 

In [5]:
def count_samples(c_list):

  samples_count = {'soil': 0,'tree': 0}

  for sample in c_list:
    if sample[1] == 0:
      samples_count['soil'] = samples_count['soil']+1
    elif sample[1] == 1:
      samples_count['tree'] = samples_count['tree']+1

  print(samples_count)

Inicializando função para separar e balancear dados. 

In [6]:
import random

def data_split(d_list, split_prop = np.array([0.8,0.1,0.1]), balance = True):

  # garantindo que a soma da divisão entre treinamento, validação e teste é 1.
  try:
    np.sum(split_prop) == 1.0
  except:
    print('split_prop sum != 1.')

  else:
    soil_list = []
    tree_list = []

    train_list = []
    val_list = []
    test_list = []

    # separando as imagens por classificação.
    for name, label in d_list:

      # carregando as imagens
      img_array = cv2.imread(os.path.join(DATADIR,name), cv2.IMREAD_COLOR)
      img_array_rgb = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)

      if label:
        tree_list.append([name,label,img_array_rgb])
      else:
        soil_list.append([name,label,img_array_rgb])

    # balanceando os dados, caso solicitado.
    if balance:
      lower = min(len(tree_list),len(soil_list))

      tree_list = tree_list[:lower]
      soil_list = soil_list[:lower]

    # definindo os valores em que as listas serão separadas.
    train_list_ts = int(len(tree_list)*split_prop[0])
    train_list_ss = int(len(soil_list)*split_prop[0])
    
    val_list_ts = train_list_ts + int(len(tree_list)*split_prop[1])
    val_list_ss = train_list_ss + int(len(soil_list)*split_prop[1])

    # dividindo em teste, treino e validação.
    train_list = tree_list[:train_list_ts] + soil_list[:train_list_ss]
    val_list = tree_list[train_list_ts:val_list_ts] + soil_list[train_list_ss:val_list_ss]
    test_list = tree_list[val_list_ts:] + soil_list[val_list_ss:]

    random.shuffle(train_list)
    random.shuffle(val_list)
    random.shuffle(test_list)

    return train_list, val_list, test_list


Dado que há imagens com falhas (pixels pretos), a função a seguir retorna a imagem com os pixels falhos tendo seus valores RGB alterados para a média de cada componente.

In [7]:
def avg_on_missing_values(bad_img, print_media = False):
  R_sum = 0
  G_sum = 0
  B_sum = 0
  k = 0

  # obtendo a média de cada componente RGB dos pixels não falhos.
  for i in range(len(bad_img)):
    for j in range(len(bad_img[i])):
      if bad_img[i,j].any() != 0:
        k = k+1

        R_sum = R_sum + bad_img[i,j,0]
        G_sum = G_sum + bad_img[i,j,1]
        B_sum = B_sum + bad_img[i,j,2]

  R_mean = R_sum/k
  G_mean = G_sum/k
  B_mean = B_sum/k

  pixel_mean = np.array([R_mean,G_mean,B_mean])

  if print_media:
    print(f'[R={pixel_mean[0]:.1f},  G={pixel_mean[1]:.1f}, B={pixel_mean[2]:.1f} ]')

  # substituindo os pixels falhos pela média obtida.
  for i in range(len(bad_img)):
    for j in range(len(bad_img[i])):
      if bad_img[i,j].any() == 0:
        bad_img[i,j] = pixel_mean.astype('uint8')
  
  return(bad_img)

Realizando o balanceamento e limpeza dos dados

*   Será utilizado a divisão dos dados de (8:1:1) para treinamento,validação e teste, respectivamente.

*   Tendo em vista que há 2 vezes mais amostras de imagens de solo do que amostras de imagens de árvores, será aplicado o balanceamento dos dados igualando o número de amostras.



In [8]:
# carregando a lista de imagens
data_list = []
for img in imgs_list:
  class_num = CATEGORIES.index(img[5:9])

  # [nome_do_arquivo, classificação]
  data_list.append([img, class_num])

print('Número de amostras antes do balanceamento:')
count_samples(data_list)

# dividindo e balanceando os dados
# [nome_do_arquivo, classificação, imagem]
train_list, val_list, test_list = data_split(data_list)

# corrigindo imagens com pixels falhos
train_list_corrected = []
val_list_corrected = []
test_list_corrected = []

for name, label, img in train_list:
  train_list_corrected.append([name, label, avg_on_missing_values(img)])

for name, label, img in val_list:
  val_list_corrected.append([name, label, avg_on_missing_values(img)])

for name, label, img in test_list:
  test_list_corrected.append([name, label, avg_on_missing_values(img)])

print('\nNúmero de amostras após o balanceamento:')
print('Treino:')
count_samples(train_list_corrected)
print('Validação')
count_samples(val_list_corrected)
print('Teste:')
count_samples(test_list_corrected)

Número de amostras antes do balanceamento:
{'soil': 4448, 'tree': 2224}

Número de amostras após o balanceamento:
Treino:
{'soil': 1779, 'tree': 1779}
Validação
{'soil': 222, 'tree': 222}
Teste:
{'soil': 223, 'tree': 223}


[Voltar ao topo](#ancora)

## Feature Engineering <a name="feat_eng"></a>

Procurando destacar a diferença de cores, sabendo que a imagem encontra-se em RGB e R é o valor da componente vermelha do pixel; G é o valor da componente verde do pixel e; B é o valor da componente azul do pixel; Os pixels serão normalizados de acordo com a seguinte equação:

$$ pixel = [\frac{R}{R+G+B}, \frac{G}{R+G+B}, \frac{B}{R+G+B} ]$$

Inicializando função de normalização dos pixels. 

In [9]:
def normalize_rgb(rgb_img, printable = False):
  # se 'printable' inicializado como 'True', a imagem da saída terá os valores entre 0 e 255
  
  rgb_img = rgb_img.astype('float32')

  # recuperando uma lista separada para cada componente RGB do pixel
  r = rgb_img[:,:,0]
  g = rgb_img[:,:,1]
  b = rgb_img[:,:,2]

  norm_img = np.zeros(rgb_img.shape, np.float32)

  if printable:
    printable_norm_img = np.zeros(rgb_img.shape, np.uint8)

  # realizando a operação de normalização para cada pixel
  for i in range(len(rgb_img)):
    for j in range(len(rgb_img[i])):

      sum = r[i,j] + g[i,j] + b[i,j]
      
      norm_img[i,j] = np.array([r[i,j]/sum, g[i,j]/sum, b[i,j]/sum])
      if printable:
        printable_norm_img[i,j] = np.array(norm_img[i,j]*255.0).astype('uint8')
        
  if printable:
    norm_img = printable_norm_img
        
  return norm_img

Normalizando as imagens

In [10]:
norm_train = []
norm_val = []
norm_test = []

for name, label, img in train_list_corrected:
  norm_train.append([name, label, normalize_rgb(img)])

for name, label, img in val_list_corrected:
  norm_val.append([name, label, normalize_rgb(img)])

for name, label, img in test_list_corrected:
  norm_test.append([name, label, normalize_rgb(img)])

Organizando as listas em um dicionário composto por DataFrames

In [11]:
import pandas as pd

processed_data_dict = {}
def list_to_dataframe(key, list_corrected):
  name_list = []
  label_list = []
  normalized_imgs_list = []
  
  aux_data_dict = {}

  for name, label, img in list_corrected:
    name_list.append(name)
    label_list.append(label)
    normalized_imgs_list.append(img)

  aux_data_dict = {'name':name_list, 'label':label_list, 'image':normalized_imgs_list}
  processed_data_dict[key] = pd.DataFrame(aux_data_dict)

list_to_dataframe('train', norm_train)
list_to_dataframe('val', norm_val)
list_to_dataframe('test', norm_test)

Exportando os dados após pré-processamento:

In [12]:
import pickle

pickle_out = open("/content/XMB/processed_data.pickle","wb")
pickle.dump(processed_data_dict, pickle_out)
pickle_out.close()

[Voltar ao topo](#ancora)

# Implementação do modelo (CNN) <a name="CNN"></a>

Importando as bibliotecas

In [13]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D 
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping, ModelCheckpoint
import pickle
import time

Inicializando dados

In [14]:
processed_data_dict = pickle.load(open("/content/XMB/processed_data.pickle","rb"))

Definindo função para separar dados entre input (X) e output (y).

In [15]:
def split_Xy(processed_data):
  X = []
  y = []

  IMG_SIZE = processed_data['image'][0].shape[0]

  for features, label in zip(processed_data['image'],processed_data['label']):
    X.append(features)
    y.append(label)

  # realizando reshape para usar como input no CNN.
  X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
  y = np.array(y)

  return X,y

X_train, y_train = split_Xy(processed_data_dict['train'])
X_val, y_val = split_Xy(processed_data_dict['val'])
X_test, y_test = split_Xy(processed_data_dict['test'])

Treinando o modelo:

*   Devido a sua eficácia em tarefas de classificação de imagens, o modelo implementado foi a Rede Neural Convolucional (CNN)
*   O algoritmo irá realizar um Grid Search entre 3 valores de *dense layers*, *layer sizes* e *conv layers*.
*   O número de épocas foi definido como 50, porém está sendo incluso um *EarlyStopping* de paciencia 5 monitorando as perdas na validação (*val_loss*).
*   O modelo definido como o melhor para cada conjunto de hyperparâmetros no Grid Search é aquele que teve a melhor acurácia na validação (*val_acc*) dentre todas as épocas.



In [16]:
from tqdm import tqdm

dense_layers = [0, 1, 2]
layer_sizes = [32, 64, 128]
conv_layers = [1, 2, 3]

total_tqdm = len(dense_layers)*len(layer_sizes)*len(conv_layers)

best_results = [0,0]

with tqdm(total=total_tqdm) as pbar:
  for dense_layer in dense_layers:
      for layer_size in layer_sizes:
          for conv_layer in conv_layers:
            
            NAME = "{}-conv-{}-nodes{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            tensorboard = TensorBoard(log_dir='/content/XMB/logs/{}'.format(NAME))

            model = Sequential()

            model.add(Conv2D(layer_size, (3,3), input_shape = X_train.shape[1:]))
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size=(2,2)))
            model.add(Dropout(0.2))

            for l in range(conv_layer-1):
              model.add(Conv2D(layer_size, (3,3)))
              model.add(Activation("relu"))
              model.add(MaxPooling2D(pool_size=(2,2)))
              model.add(Dropout(0.2))

            model.add(Flatten())

            for l in range(dense_layer):  
              model.add(Dense(layer_size))
              model.add(Activation("relu"))
              model.add(Dropout(0.2))

            model.add(Dense(64))
            model.add(Activation("relu"))
            model.add(Dropout(0.2))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            model.compile(loss="binary_crossentropy",
                        optimizer="adam",
                        metrics=['accuracy'])

            model.fit(X_train, y_train, 
                      batch_size=32,
                      epochs = 50,
                      verbose=0,
                      validation_data = (X_val,y_val),
                      callbacks=[tensorboard,
                                 EarlyStopping(monitor='val_loss', 
                                              mode='min', 
                                              patience=5,
                                              verbose=0,
                                              restore_best_weights=False),
                                 ModelCheckpoint('/content/XMB/best_model.h5',
                                                monitor='val_accuracy',
                                                mode='max',
                                                verbose=0,
                                                save_best_only=True)])
            
            # avaliando o modelo
            saved_model = load_model('/content/XMB/best_model.h5')
            results = saved_model.evaluate(X_test, y_test, batch_size=32, verbose=0)

            # verificando se o modelo atual é o melhor
            cur_loss, cur_acc = results
            best_loss, best_acc = best_results 

            if cur_acc>best_acc or (cur_acc == best_acc and cur_loss<best_loss): 
              best_results = results

              param_dct = {'dense_layer': dense_layer,
                          'layer_size': layer_size,
                          'conv_layer': conv_layer}
              
              model_dct = {'name':NAME,
                            'model':saved_model,
                            'param':param_dct,
                            'results':results}

            pbar.update(1)

print("Melhor modelo encontrado:\n")
print(f"acurácia(accuracy): {model_dct['results'][1]}")
print(f"perdas(loss): {model_dct['results'][0]}\n")
print("Parâmetros:")
print(model_dct['param'])
model_dct['model'].save('/content/XMB/model/trees-vs-soil-CNN.h5')

100%|██████████| 27/27 [12:51<00:00, 28.56s/it]

Melhor modelo encontrado:

acurácia(accuracy): 0.9192824959754944
perdas(loss): 0.2378302961587906

Parâmetros:
{'dense_layer': 2, 'layer_size': 32, 'conv_layer': 3}


Exportando modelo e dados do Tensorboard.

In [17]:
!zip -r /content/XMB/model.zip /content/XMB/model

  adding: content/XMB/model/ (stored 0%)
  adding: content/XMB/model/trees-vs-soil-CNN.h5 (deflated 25%)


In [18]:
!zip -r /content/XMB/logs.zip /content/XMB/logs

  adding: content/XMB/logs/ (stored 0%)
  adding: content/XMB/logs/1-conv-64-nodes1-dense-1685025865/ (stored 0%)
  adding: content/XMB/logs/1-conv-64-nodes1-dense-1685025865/train/ (stored 0%)
  adding: content/XMB/logs/1-conv-64-nodes1-dense-1685025865/train/events.out.tfevents.1685025865.4d1b732e184a.169.24.v2 (deflated 75%)
  adding: content/XMB/logs/1-conv-64-nodes1-dense-1685025865/validation/ (stored 0%)
  adding: content/XMB/logs/1-conv-64-nodes1-dense-1685025865/validation/events.out.tfevents.1685025868.4d1b732e184a.169.25.v2 (deflated 76%)
  adding: content/XMB/logs/3-conv-64-nodes1-dense-1685025898/ (stored 0%)
  adding: content/XMB/logs/3-conv-64-nodes1-dense-1685025898/train/ (stored 0%)
  adding: content/XMB/logs/3-conv-64-nodes1-dense-1685025898/train/events.out.tfevents.1685025898.4d1b732e184a.169.28.v2 (deflated 80%)
  adding: content/XMB/logs/3-conv-64-nodes1-dense-1685025898/validation/ (stored 0%)
  adding: content/XMB/logs/3-conv-64-nodes1-dense-1685025898/validati

In [19]:
files.download('/content/XMB/logs.zip')
files.download('/content/XMB/model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[Voltar ao topo](#ancora)